This notebook was run in Julia 1.6. The following Julia packages are used in this note book.

DifferentialEquations, Flux, DiffEqFlux, Optim, DiffEqSensitivity, Plots, OrdinaryDiffEq, Zygote, StaticArrays, LinearAlgebra, BenchmarkTools, PaddedViews

These packages can be installed using the following commands:
```
using Pkg;
Pkg.add(["OrdinaryDiffEq","DifferentialEquations", "Flux", "DiffEqFlux", "Optim", "DiffEqSensitivity", "Plots", "Zygote", "StaticArrays", "BenchmarkTools", "PaddedViews","NLopt","BlackBoxOptim", "DiffEqBase"]);
```

In [ ]:
gr(show = :ijulia)

In [ ]:
using DifferentialEquations, Flux, DiffEqFlux, DiffEqSensitivity, Plots, OrdinaryDiffEq, Optim, Zygote, StaticArrays, LinearAlgebra, BenchmarkTools, PaddedViews, PlotThemes
using NLopt,BlackBoxOptim, DiffEqBase, SparseArrays

Defining the constants needed for all leakage optimizations.

In [ ]:

const σ0 = Hermitian(Complex{Float64}[1 0; 0 1]);
const σx = Hermitian(Complex{Float64}[0 1; 1 0]);
const σy = Hermitian(Complex{Float64}[0 -im; im 0]);
const σz = Hermitian(Complex{Float64}[1 0; 0 -1]);
int(x) = floor(Int, x)
const Ωmax = 4.0;
const Jmax = 1.0;

Depending on the targeted gate the targeted rotation in the decomposed Hamiltonian is different

In [ ]:
const Utarget = σ0#exp(im*π/2*σz); #exp(im*π/2*σx);
# for ZZ gate with 3 nearest neighbors use:
const Utarget1 = exp(im*π/4*σz);
const Utarget2 = exp(3*im*π/4*σz);
# for ZZ gate with 4 nearest neighbors use:
const Utarget0 = σ0;
const Utarget1 = exp(im*π/2*σz);
const Utarget2 = exp(im*π/1*σz);

In the next section the blocks define the constants needed for each different vertex type (2,3, or 4 nearest neighbors).
First is 2 neighborgrs

In [ ]:
# constants for 2 nearest neighbors
const T=4.5*π;
const steepness = 2*T
const u0 = Complex{Float64}[σ0; σ0; σ0; σ0*0;σ0*0;σ0*0;[0.0 0.0]];
const numberF=40
pinitial=ones(numberF)./numberF.*7;
const III = kron(σ0,kron(σ0,σ0));
const IIX = kron(σ0,kron(σ0,σx));
const IIY = kron(σ0,kron(σ0,σy));
const IZZ = kron(σ0,kron(σz,σz));
const ZIZ = kron(σz,kron(σ0,σz));
tspan = (0.0, T);
const U0 = III ;
const UtargetAll = III;#exp(im*π/4*IIX); exp(im*π/4*(ZIZ+IZZ))# Choose desired gate 

In [ ]:
# constants for 3 nearest neighbors
const IIII = kron(σ0,kron(σ0,kron(σ0,σ0)));;
const IIIX = kron(σ0,kron(σ0,kron(σ0,σx)));
const IIIY = kron(σ0,kron(σ0,kron(σ0,σy)));
const ZIIZ = kron(σz,kron(σ0,kron(σ0,σz)));
const IIZZ = kron(σ0,kron(σ0,kron(σz,σz)));
const IZIZ = kron(σ0,kron(σz,kron(σ0,σz)));
const T=6*π;
const u0 = Complex{Float64}[σ0; σ0; σ0; σ0; 0*σ0; 0*σ0; 0*σ0; 0*σ0; 0 0]
tspan = (0.0, T);
const U0 = IIII;
const UtargetAll = exp(im*pi/4*IIIX); #exp(im*π/4*(IIZZ+IZIZ+ZIIZ));
pinitial=ones(numberF)./numberF.*7;

In [ ]:
# constants for 4 nearest neighbors
const IIIII = kron(σ0,kron(σ0,kron(σ0,kron(σ0,σ0))));
const IIIIX = kron(σ0,kron(σ0,kron(σ0,kron(σ0,σx))));
const IIIIY = kron(σ0,kron(σ0,kron(σ0,kron(σ0,σy))));
const IZIIZ = kron(σ0,kron(σz,kron(σ0,kron(σ0,σz))));
const IIIZZ = kron(σ0,kron(σ0,kron(σ0,kron(σz,σz))));
const IIZIZ = kron(σ0,kron(σ0,kron(σz,kron(σ0,σz))));
const ZIIIZ = kron(σz,kron(σ0,kron(σ0,kron(σ0,σz))));
const T=10*π;
const u0 = Complex{Float64}[σ0; σ0; σ0; σ0; σ0;0*σ0;0*σ0;0*σ0;0*σ0;0*σ0;  0 0];
const numberF=100;
tspan = (0.0, T);
const U0 = IIIII;
const UtargetAll = exp(im*pi/4*IIIIX);
pinitial=ones(numberF)./numberF.*7;

Schrodinger equation \
For each number of nearest neighbors there is a different Schrodinger equation.\
First is the 2 vertices, then 3 vertices and finally 4 vertices as neighbor version.


In [ ]:
# Schrodinger equation for 2 nearest neighbors
# all these differential equations calculate both the evolution operator as well as the first order magnus expansion and the integral of how much extra power over max power is used.
function schrodinger_f(u, p, t)
    @views @inbounds U1 = u[1:2, 1:2]; # each seperate evolution operator in the commuting su(2) spaces
    @views @inbounds U2 = u[3:4, 1:2];
    @views @inbounds U3 = u[5:6, 1:2];
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)# construction of the instentanious value of Ωx and Ωy
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+p[2*i-1]*sina
        Ωy=Ωy+p[2*i]*sina
    end 
    local H1 = Hermitian(Ωx/2*σx+Ωy/2*σy+Jmax/2*σz);
    local H3 = Hermitian(Ωx/2*σx+Ωy/2*σy-Jmax/2*σz);
    local H2 = Hermitian(Ωx/2*σx+Ωy/2*σy);
    local HE = Hermitian(Jmax/2*σz);
    local power=relu(abs((Ωx^2+Ωy^2))-Ωmax^2)
    return [-(im*H1)*U1;-(im*H2)*U2;-(im*H3)*U3 ;U1'*HE*U1;U2'*HE*U2; U3'*HE*U3; [power 0]]
end
ode_f = ODEProblem(schrodinger_f, u0, tspan, pinitial);
# loss function is a weighted sum of the trace fidelity, integrated amplitude over max and norm of the magnus expansion generator
function cost(p)
    local ode_sol = solve(ode_f, BS5(), p=Complex{Float64}.(p), abstol=1.0e-7, reltol=1.0e-7) 
    local usol = last(ode_sol)
    @views @inbounds local Ugate1 = usol[1:2, 1:2]; 
    @views @inbounds local Ugate2 = usol[3:4, 1:2]; 
    @views @inbounds local Ugate3 = usol[5:6, 1:2]; 
    @views @inbounds local ℰ1 = usol[7:8, 1:2];
    @views @inbounds local ℰ2 = usol[9:10, 1:2];
    @views @inbounds local ℰ3 = usol[11:12, 1:2];
    @views @inbounds local intPower=usol[13,1]
    # for a ZZ gate replace (Ugate2*Utarget') with (Ugate2) in the loss function in the line below.
    @inbounds local loss = abs(intPower)+abs(1.0-((tr(Ugate1*Utarget')/2)))+abs(1.0-((tr(Ugate2*Utarget')/2)))+abs(1.0-((tr(Ugate3*Utarget')/2)))+0.1*((norm(ℰ1))^2+(norm(ℰ2))^2+(norm(ℰ3))^2)
    return loss
end

plot_callback = function (p,l) #input current params p and cost l; stores and plots history of cost function throughout optimization
    append!(cost_history,l)
    plot(log10.(cost_history), xtickfont=18, ytickfont=18, legendfont=18);
    return false
end

In [ ]:
# Schrodinger equation for 3 nearest neighbors
function schrodinger_f(u, p, t)
    @views @inbounds U1 = u[1:2, 1:2];
    @views @inbounds U2 = u[3:4, 1:2];
    @views @inbounds U3 = u[5:6, 1:2];
    @views @inbounds U4 = u[7:8, 1:2];
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+p[2*i-1]*sina
        Ωy=Ωy+p[2*i]*sina
    end
    local H1 = Hermitian(Ωx/2*σx+Ωy/2*σy+Jmax/4*σz);
    local H2 = Hermitian(Ωx/2*σx+Ωy/2*σy+3*Jmax/4*σz);
    local H3 = Hermitian(-Ωx/2*σx-Ωy/2*σy-3*Jmax/4*σz);
    local H4 = Hermitian(-Ωx/2*σx-Ωy/2*σy-Jmax/4*σz);
    local HE = Hermitian(Jmax/4*σz);
    local power=relu(abs((Ωx^2+Ωy^2))-Ωmax^2)
    return [-(im*H1)*U1;-(im*H2)*U2;-(im*H3)*U3;-(im*H4)*U4; U1'*HE*U1; U2'*HE*U2; U3'*HE*U3; U4'*HE*U4; [power 0]]
end

ode_f = ODEProblem(schrodinger_f, u0, tspan, pinitial);

function cost(p)
    local ode_sol = solve(ode_f, BS5(), p=Complex{Float64}.(p), abstol=1.0e-7, reltol=1.0e-7) 
    local usol = last(ode_sol)
    @views @inbounds local Ugate1 = usol[1:2, 1:2]; 
    @views @inbounds local Ugate2 = usol[3:4, 1:2];  
    @views @inbounds local Ugate3 = usol[5:6, 1:2];  
    @views @inbounds local Ugate4 = usol[7:8, 1:2];
    @views @inbounds local E1 = usol[9:10, 1:2];
    @views @inbounds local E2 = usol[11:12, 1:2];
    @views @inbounds local E3 = usol[13:14, 1:2];
    @views @inbounds local E4 = usol[15:16, 1:2];
    @views @inbounds local intPower=usol[17,1]
    # For ZZ gate replace line below with: "@inbounds local loss = abs(1.0-((tr(Ugate1*Utarget1')/2)))+abs(1.0-((tr(Ugate2*Utarget2')/2)))+abs(1.0-((tr(Ugate3*Utarget2)/2)))+abs(1.0-((tr(Ugate4*Utarget1)/2)))+1*(0.3*(norm(E1))^2+0.3*(norm(E2))^2+0.3*(norm(E3))^2+0.3*(norm(E4))^2) "
    @inbounds local loss = abs(intPower)+ abs(1.0-((tr(Ugate1*Utarget')/2)))+abs(1.0-((tr(Ugate2*Utarget')/2)))+abs(1.0-((tr(Ugate3*Utarget)/2)))+abs(1.0-((tr(Ugate4*Utarget)/2))^2)+6*(0.3*(norm(E1))^2+0.3*(norm(E2))^2+0.3*(norm(E3))^2+0.3*(norm(E4))^2)  
    return loss
end

plot_callback = function (p,l) #input current params p and cost l; stores and plots history of cost function throughout optimization
    append!(cost_history,l)
    plot(log10.(cost_history), xtickfont=18, ytickfont=18, legendfont=18);
    return false
end

In [ ]:
# Schrodinger equation for 4 nearest neighbors
function schrodingerf(u, p, t)
    @views @inbounds U1 = u[1:2, 1:2];
    @views @inbounds U2 = u[3:4, 1:2];
    @views @inbounds U3 = u[5:6, 1:2];
    @views @inbounds U4 = u[7:8, 1:2];
    @views @inbounds U5 = u[9:10, 1:2];
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+p[2*i-1]*sina
        Ωy=Ωy+p[2*i]*sina
    end
    local H1 = Hermitian(Ωx/2*σx+Ωy/2*σy+Jmax*σz);
    local H2 = Hermitian(-(Ωx/2*σx+Ωy/2*σy)-Jmax*σz);
    local H3 = Hermitian(Ωx/2*σx+Ωy/2*σy-Jmax/2*σz);
    local H4 = Hermitian(Ωx/2*σx+Ωy/2*σy+Jmax/2*σz);
    local H5 = Hermitian(Ωx/2*σx+Ωy/2*σy);
    local HE = Hermitian(Jmax/4*σz);
    local power=relu(abs((Ωx^2+Ωy^2))-Ωmax^2)
    return [-(im*H1)*U1;-(im*H2)*U2;-(im*H3)*U3;-(im*H4)*U4;-(im*H5)*U5; U1'*HE*U1; U2'*HE*U2; U3'*HE*U3; U4'*HE*U4; U5'*HE*U5; [power 0]]
end

odef = ODEProblem(schrodingerf, u0, tspan, pinitial);

function cost(p)
    local ode_sol = solve(odef, BS5(), p=Complex{Float64}.(p), abstol=1.0e-7, reltol=1.0e-7) 
    local usol = last(ode_sol)
    @views @inbounds local Ugate1 = usol[1:2, 1:2]; 
    @views @inbounds local Ugate2 = usol[3:4, 1:2]; 
    @views @inbounds local Ugate3 = usol[5:6, 1:2]; 
    @views @inbounds local Ugate4 = usol[7:8, 1:2]; 
    @views @inbounds local Ugate5 = usol[9:10, 1:2]; 
    @views @inbounds local E1 = usol[11:12, 1:2];
    @views @inbounds local E2 = usol[13:14, 1:2];
    @views @inbounds local E3 = usol[15:16, 1:2];
    @views @inbounds local E4 = usol[17:18, 1:2];
    @views @inbounds local E5 = usol[19:20, 1:2];
    @views @inbounds local intPower=usol[21,1]
    # For ZZ gate replace line below with: "@inbounds local loss = 0.03*abs(intPower)+ abs(1.0-((tr(Ugate1*Utarget2')/2)))+abs(1.0-((tr(Ugate2*Utarget2)/2)))+abs(1.0-((tr(Ugate3*Utarget1)/2)))+abs(1.0-((tr(Ugate4*Utarget1')/2)))+abs(1.0-((tr(Ugate5*Utarget0)/2)))+0.3*((norm(E1))^2+(norm(E2))^2+(norm(E3))^2+(norm(E4))^2+(norm(E5))^2)"
    @inbounds local loss = 0.03*abs(intPower)+abs(1.0-((tr(Ugate1*Utarget')/2)))+abs(1.0-((tr(Ugate2*Utarget)/2)))+abs(1.0-((tr(Ugate3*Utarget')/2)))+abs(1.0-((tr(Ugate4*Utarget')/2)))+abs(1.0-((tr(Ugate5*Utarget)/2)))+0.6*((norm(E1))^2+(norm(E2))^2+(norm(E3))^2+(norm(E4))^2+(norm(E5))^2)
    return loss

end

plot_callback = function (p,l) #input current params p and cost l; stores and plots history of cost function throughout optimization
    append!(cost_history,l)
    plot(log10.(cost_history), xtickfont=18, ytickfont=18, legendfont=18);
    return false
end

Optimizatoin\
The initial parameters are optimized using Radam and BFGS optimizers (more optimizations loops can be added if needed).\
The cost history is initialized and reset first then the while running DiffEqFlux it is possible to see the costfunction value while the optimizer is running.

In [ ]:
cost_history=[] 

In [ ]:
@time dnn_sol=DiffEqFlux.sciml_train(cost,pinitial, RADAM(0.002),cb=plot_callback, progress=true,maxiters=20);

In [ ]:
@time dnn_sol2=DiffEqFlux.sciml_train(cost,dnn_sol.minimizer, BFGS(initial_stepnorm=0.001),cb=plot_callback,maxiters=50);

Next we plot the resulting pulse (works for any number of neighboring vertices)

In [ ]:
soll=2vertex_Jfluct_XGate;#dnn_sol2.minimizer for direct optimization result plotting
function Omegax(t)
    Ωx =0
    p1=soll
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+p1[2*i-1]*sina
    end  
    return Ωx
end 

function Omegay(t)
    Ωy = 0
    p1=soll
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωy=Ωy+p1[2*i]*sina
    end  
    return Ωy
end

maxpower(t)=Ωmax;
minpower(t)=-Ωmax;
power(t)=sqrt(Omegax(t)^2+Omegay(t)^2);
plot([Omegax Omegay  power maxpower minpower] ,0,T) ;

Calculating the resulting infidelity of the optimized pulses and plotting them \
The funciton are different depending on the number of nearest neighbor vertices.

In [ ]:
# Trace infidelity function for 2 nearest neighbors
soll=2vertex_Leak_Xgate# or (dnn_sol2.minimizer) this is the weights of the pulse tested here
# See below for the weights of the gates presented in the paper 

numberF=length(soll) #in case switching between optimizations here is a good place to check the number of Sin series terms taken.

#The schrodinger equation again but now the extra function p is used to see how much J can fluctuate
function schrodinger_ϵ(u,p,t)
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+soll[2*i-1]*sina
        Ωy=Ωy+soll[2*i]*sina
    end
    local H = Hermitian(Ωy/2*IIY+Ωx/2*IIX+(1+p)*Jmax/4*(ZIZ+IZZ));    
    return -im*(H)*u 
end

prob = ODEProblem(schrodinger_ϵ, U0, tspan)
# infidelity function returns the trace fidelity as a function of offset in J
function infidelity(ϵ)
    sol = solve(prob, BS5(), p=ϵ, abstol=1e-9, reltol=1e-9)
    @views Ugate = last(sol)[1:8,1:8]
    return abs(1.0-((tr(UtargetAll'*Ugate)/8)))
end 

In [ ]:
# Trace infidelity function for 3 nearest neighbors
soll=3vertex_Leak_Xgate;
function schrodinger_ϵ(u,p,t)
    @views @inbounds U=u[1:16,1:16];
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+soll[2*i-1]*sina
        Ωy=Ωy+soll[2*i]*sina
    end
    local H = Hermitian(Ωy/2*IIIY+Ωx/2*IIIX+(1+p)*Jmax/4*(ZIIZ+IZIZ+IIZZ));    
    return -im*(H)*U
end

prob = ODEProblem(schrodinger_ϵ, U0, tspan)

function infidelity(ϵ)
    sol = solve(prob, BS5(), p=ϵ, abstol=1e-9, reltol=1e-9)
    @views Ugate = last(sol)[1:16,1:16]
    return (1.0-abs((tr(UtargetAll'*Ugate)/16))^2)
end 

In [ ]:
# Trace infidelity function for 4 nearest neighbors
soll=4vertex_Leak_Xgate;
function schrodinger_ϵ(u,p,t)
    @views @inbounds U = u[1:32, 1:32];
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+soll[2*i-1]*sina
        Ωy=Ωy+soll[2*i]*sina
    end
    local H = Hermitian(Ωx/2*IIIIX+Ωy/2*IIIIY+(1+p)*Jmax/4*(IZIIZ+IIZIZ+IIIZZ+ZIIIZ));  
    return -im*H*U
end

prob = ODEProblem(schrodinger_ϵ, U0, tspan)
    
function infidelity(ϵ)
    sol = solve(prob, BS5(), p=ϵ, abstol=1e-9, reltol=1e-9)
    @views Ugate = last(sol)[1:32,1:32]
    return  abs(1.0-((tr(UtargetAll'*Ugate)/32))^2)
end 

In [ ]:
infidelity(0.000)

We can also plot the infidelity as a function of this offset in J

In [ ]:
Plots.plot([infidelity x->x^2 x->x^4], 0.001, 1, yaxis=:log10,xaxis=:log10,ylims=(1e-4,1e-0),ylabel="Infidelity",xlabel=L"\delta \Delta",xguidefontsize=18,yguidefontsize=18,xtickfont=13, ytickfont=18, legendfont=18,legend=:topleft);

The weights for the optimized pulses resulting in X, ZZ, and Identity gates robust to J fluctuations are reported below: 

In [ ]:
2vertex_Jfluct_XGate=[0.5969418499774933,1.016873885394134,-0.05397291925328053,0.14680088294945107,0.8538519493681225,0.4375049952631781,0.5918169334679045,0.06384220045805185,-0.6274277857669683,0.187412141579835,-0.021856787467344284,-0.029552714635155936,0.4394209510914522,0.09504486661604396,0.10536747252466803,-0.05130076103203475,-0.62322311712364,0.15541143587301298,0.004628406070746844,-0.034898160314177376,0.42626815068648005,0.1006700888549711,0.1295955631974517,-0.1213779633623996,0.07598396317525759,0.008113597287073827,0.09371967046170353,-0.020485099585731345,0.08470124391482171,0.08334652039869356,0.06448559958697644,0.026131685057828365,0.08393086140241525,0.08791895261618692,0.06812158362211446,0.040528208373944524,0.08189528706388309,0.08606330932221479,0.08192317739341706,0.05026057937593829,0.07539909197489268,0.08886767390353349,0.07430147954239318,0.051080166572398146,0.0840287206224374,0.08336247613647,0.08483543263421665,0.056596602495019147,0.07865500321799965,0.0843154079494689,0.08166349262252193,0.06357663671189781,0.08315133573319318,0.08648913075707669,0.08445462040152166,0.06365255629121733,0.08109821770711305,0.08629668609586728,0.08478217359750888,0.06783530042485779,0.08418593990647916,0.0846720370655957,0.08676865747867094,0.06644133131479252,0.08305975209264692,0.085717439530485,0.08650215767713261,0.07082976794419203,0.08350315927455411,0.08652301737410126,0.08715542145100436,0.0731170057269648,0.08359136994904519,0.08750876371893011,0.08659464073490143,0.07418610248653162,0.08800208011649907,0.08513999928713964,0.09349299509769246,0.06931906202801405];
2vertex_Jfluct_ZZGate=[-0.36571823158094735,-0.001333013914235898,0.16677512157680666,-0.03632719541378748,0.7011434129812812,0.11125161229327704,-0.0792800959685207,0.03778261046918527,0.2622254236858908,-0.00709682471666713,0.34897193954791644,0.030991160422975506,0.29814545882454974,-0.0007905741790797937,0.27920523674118225,-0.0067447716210743015,0.2424808312360644,-0.0035328195338063468,0.21265175443511752,-0.016159025062971368,0.19061104387653438,-0.017560172768906052,0.16955083534479715,-0.017035223893632406,0.15502032886445866,-0.016553799049230506,0.14523603324374373,-0.011977211745649216,0.13848313050514013,-0.005358546788470253,0.1363398146186416,0.00319377845304713,0.13708890831503673,0.013802335178021924,0.14053594536728076,0.02608164295081822,0.146747912586052,0.039784423356485896,0.15471618984207428,0.05537751517536889];
2vertex_Jfluct_IdGate=[1.005157430892721,-0.5595673192773252,0.8719601110539801,0.823282041543517,-0.671462130496905,-0.19945658896682006,-0.10698027337000106,0.3650033524497814,-0.14062148347991915,-0.037288605732536825,-0.446391257049568,0.22867207286513583,-0.17074023449495246,-0.13607045167241105,-0.17952693445555773,-0.049249188512514176,-0.2585448701737883,-0.01691607549706597,-0.21520880372085505,-0.06915694335130954,-0.20918216935354944,-0.05170824910596312,-0.1991802336177129,-0.08626303071579947,-0.1636390872171348,-0.06743714926811889,-0.18788299980346987,-0.0344129496925628,-0.16639423615253507,-0.05825511927040076,-0.11540642612122838,-0.049326779934785944,-0.12174174077443582,-0.0022048844948572154,-0.11225912676274889,0.0006008416378550627,-0.061497903244174396,0.003880006896952503,-0.040523076517573746,0.0438131659956317];
3vertex_Jfluct_XGate=[0.35038563669520156,0.4961343210908786,-0.11794307091882723,0.23182126477714055,0.5218924273432041,0.6785234972627606,0.8507308123143953,-0.0380730459311487,-0.35932273012058785,-0.4164541989973526,0.1759079055889358,-0.12054004883014886,0.2091318031091253,0.09090914761902913,0.5992395031981486,-0.2396733926341687,0.6287634187990777,0.4055660879793936,-0.44009704258780946,0.3395281476484513,0.27789539683150066,0.06112420734176258,-0.07628417541810487,-0.035176148978757094,-0.39774932622017223,0.65101904048455,0.11870964711191939,0.14256078419935517,0.22351914313284066,-0.0812818353116749,0.18176234692760115,-0.003789432442364105,-0.1688420151846478,0.03854163997109439,0.10967712980453352,0.07036672599594582,-0.05909473882015691,0.16309038300268738,-0.0830395394202174,0.05844268433216573,-0.01688841979856584,0.15546364405441188,0.13433734049364268,0.15564936392033774,0.04201720579239161,0.08697704174388586,0.0953823496893598,0.07375729475360442,0.08688392328117533,0.11471470203742536,0.12243390733668184,0.0660838905297808,0.053865944138484695,0.09018034560363017,0.09637994458103433,0.09535270127152876,0.12757069423926387,0.07935829763486202,0.15568938155316783,0.0429854124765228];
3vertex_Jfluct_ZZGate=[0.3988246606556051,0.41778257624062104,0.21362396930973337,0.218242833596269,0.44162569122029327,0.46277747443999834,0.31571549307230845,0.3527893212135642,-0.2894206923296223,-0.29832468467563544,-0.3059306131044627,-0.2895704507233592,0.0034607674665083706,0.03300900724371419,-0.03989088967847648,0.005974508372108571,0.5915311292446687,0.6274165176767952,0.7808230374999333,0.8299383758079457,0.18582355104428286,0.19563151780122634,0.1462063142567737,0.1568156310880699,0.3514022857423705,0.3298055521771941,-0.02341761755651541,-0.01985895063994466,0.2900727584309058,0.2966056054504697,0.22314081704379737,0.21415334421579588,0.3549946058349082,0.3377320394328764,-0.030017662885797696,-0.017869765496952884,0.13200579499906132,0.12938537298980793,-0.24743356221930773,-0.23808361027294275,0.0019700184966250666,0.010167415603774991,-0.23269610067455912,-0.22518961035512114,0.02371167866242519,0.03882992391422963,-0.17773695384284485,-0.16321108194715614,0.06049975929217061,0.07836351007388427,-0.160401364271411,-0.14839199659434132,0.06718366157219549,0.09066409942240515,-0.11030482719280031,-0.09917497591925457,0.11591063654185275,0.14704573284912592,-0.017384908548651396,-0.01593511090195962];
3vertex_Jfluct_IdGate=[0.38791537539802323,0.38981568794146176,0.22968725298280657,0.23023174673826083,0.20250915079951962,0.20219404049515294,0.924430795945179,0.9323401700989737,0.7333785518257768,0.7340856982743894,-0.1578053198777788,-0.15976079941214638,0.3467204533928789,0.34301336862212173,-0.3175392112976298,-0.3115729437052195,0.37863694459008673,0.3676806524249207,0.2984811421996303,0.2897627237198275,0.21974618855438185,0.21825517485713264,0.12855782604846566,0.12321464811609036,-0.56011606037668,-0.5494580151570773,0.2011027803656497,0.1956934803822203,0.31854420023959673,0.31609314482874146,0.10201874676994457,0.10120483273677268,0.22799806862682534,0.22634834222356343,0.14171356893431236,0.1360960606184942,0.19994329580236572,0.20233795449096478,0.1225292041897361,0.12076375719497892,0.19964040938018732,0.1996225398952712,0.11100987002333813,0.10892649515055977,0.09103848556011146,0.09051782012691588,0.010155946149894932,0.009637079771003589,0.009548593421849537,0.00803647706762383,-0.05708115030514857,-0.056664177691442975,0.00957902688649698,0.010445864807061993,-0.011255769926982587,-0.0111649559350316,0.09007483174479533,0.09255449669968341,0.09948356186282861,0.10164798768470984];
4vertex_Jfluct_XGate=[0.6088036194653351,0.5625624613039006,0.09353035614830545,0.17677790838381097,0.2115704187318568,0.2948945143002068,0.0030441979760588863,0.04335036082166268,0.20149132063199798,0.18967683635318613,-0.07299324879104879,0.1713528855573257,0.03247955514670058,-0.04462342646794796,-0.008619176991434347,0.29321965579918025,0.1935702771766314,0.21023972268269484,-0.12717752083946,0.10584697215724787,0.35886585573567653,0.03887525437914743,0.02180712156673791,-0.07588278473861886,-0.2111537072489986,0.05874492022551446,-0.059553596179103066,-0.053913905718379215,0.09181985969025541,0.06158214985341132,0.08851649018302762,-0.06464596118613149,0.1173153742190514,0.06216753836147018,0.06611890828791396,0.009075499054500705,0.19714335696789456,0.05585036245163645,0.4182105184940509,-0.006606713498960647,-0.048158838621415415,0.055849538336512995,-0.13555625272046573,-0.07251404023860634,0.1263771153086781,0.16171029296368694,0.054904971912905144,-0.027785828542761667,-0.10398777622165119,-0.24709218679931563,0.222891867444315,0.04315255422702357,0.1798681971720344,0.11507900972478868,0.12428478244177857,0.00799781212976166,0.08326713133867575,0.0368202415517648,0.09126436544932438,0.0420319101044199,0.06514604106779162,0.07428187526244251,0.09257059769608715,0.0749676893186538,0.09578418735002986,0.039905853671507806,0.09280398621516898,0.02383177982897118,0.08766893861300543,0.07453776134029097,0.102217613713473,0.06459613236320695,0.10452211239288241,0.07800454051333015,0.0854084742627346,0.04702878007890188,0.05874419249506316,0.06850507240071003,0.07124435680943209,0.06207728641327309,0.07623435075029572,0.08114770324709192,0.09423772638906093,0.04668976883103459,0.09088190469103552,0.05029749046128404,0.0867389376458327,0.03748202430847237,0.07014067965789961,0.08003931937328444,0.09986129949487665,0.07225622264997177,0.09232208613570841,0.05457319497475173,0.06378942321723749,0.045385955753894876,0.06439495724949668,0.08904218101555178,0.09331760747356953,0.06945686840132113];
4vertex_Jfluct_ZZGate=[0.6500003329928746,0.6499993266489362,0.10272632354484953,0.10272673296188325,0.18076264648112167,0.18076193404541457,0.03751649409792842,0.0375166395804291,0.2923681249864528,0.29236905092739507,0.054997426610605596,0.05499727341297389,0.19831778148432566,0.19831790734639504,0.06285819922259717,0.06286009316441442,0.07208027618317074,0.07207743323711585,0.254708439914299,0.2547120466936859,0.09181917689309868,0.09181671910858229,-0.15653655433553104,-0.15653884892345293,0.16605152964667436,0.16604946981316623,0.21911215124527367,0.2191158663085846,0.09693699963866392,0.09693840091942879,0.20752936186860155,0.20752973657117066,0.2153620452141305,0.21535874951057374,0.07259402201038243,0.07259234975432108,-0.058246481238794376,-0.058246139256276835,0.42249327972780076,0.42249403025902166,0.47751197873226897,0.4775096972872445,-0.1959183938153924,-0.19591959205728152,0.10939133927265647,0.10939110032195001,-0.12294871677230731,-0.1229500920122568,-0.016595696770599466,-0.01659668359139492,0.014393125857375334,0.014393155905194572,0.042991957113136255,0.04299450786395257,0.07064732008951352,0.07064859576639373,0.07043115255010256,0.07043093849819677,0.05978910885990835,0.05978959408925216,0.06973701480822626,0.06973650547027511,0.048525057404976465,0.04852540855552585,0.06534523987426251,0.065345875457311,0.05218965193577525,0.052190247828203105,0.051988111791394155,0.05198847406913278,0.0585379857548519,0.05853786110689567,0.054678516179182214,0.05467849961990641,0.05092102616585613,0.05092141391574893,0.05441675500163089,0.054417338222948086,0.05870638024992757,0.05870669990195189,0.05722976834140094,0.0572299450749788,0.05344237685580567,0.05344285634125408,0.058060285828756955,0.05806068720163542,0.055829705270378296,0.05582974494786371,0.0612687470488477,0.061269000110517356,0.0640778342153152,0.06407816465951094,0.06418648860320239,0.0641867524110252,0.06569928681075934,0.06569938209701177,0.06700475921070538,0.06700468286300287,0.06682739637450331,0.06682730999220414];
4vertex_Jfluct_IdGate=[1.120165945954149,1.1201636210708392,0.32940252599857245,0.32944586774406337,0.2767719427400714,0.27675995605323556,0.051687519559869374,0.051655727884689355,0.39444638266833193,0.3944534900571005,0.2380062643302721,0.23801183411066298,0.06675441486205194,0.06676809913211905,0.13571131711257736,0.1357213547034654,-0.1387001947966841,-0.13867454689710826,0.3958945717222761,0.3959128122035427,0.10725469977976651,0.1072535037538011,-0.15579186772145512,-0.1558107704124546,0.09571164783279655,0.0957280980166977,-0.011763851322219413,-0.01182181328848873,-0.44246650414296324,-0.44244569814049944,0.09135573369400628,0.0913255759650186,0.164725888340265,0.1647933342517004,0.1450450655794337,0.14503822204970562,0.16107792445402797,0.16110435854326385,0.3127786022981952,0.3127598688818747,0.4424716553261269,0.4423823684303015,0.08863364559716985,0.08870907586547871,-0.18843790903519714,-0.18836806475787451,0.03835097348810343,0.03838924227364093,0.037993981095387375,0.037974183714388605,-0.04159666108943185,-0.041631576495647526,0.10274324407749995,0.1027760601089072,0.1976618378845249,0.197613392397795,0.15737613719720991,0.15736978053437936,0.048352093426515766,0.048327242969220974,0.04096419179529047,0.04093798570068632,0.051776700557022236,0.05179529791417072,0.07328744395657473,0.0732494111403728,0.06604461405909699,0.06600721217213983,0.0521997103579029,0.05217639678973273,0.05850247168608009,0.05848192329450597,0.12089639782432007,0.12087030675087199,0.053381524249093364,0.053380452115232166,0.07204333075337162,0.0720289822920355,0.04998811211015563,0.049973743882166397,0.07005699084655428,0.07004812393618905,0.02559350592508561,0.02559165138678568,0.048493552700279184,0.04848872456706085,0.04986724152714602,0.04986776760970606,0.07674484000303673,0.07673722548631327,0.04109708773853827,0.04108135528208832,0.08273183559115427,0.08271135137617376,0.06411246119314103,0.06410339757331272,0.07473106209677015,0.07471024211718741,0.03829453550181139,0.03827583039387409];
